In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
import re
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pickle

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Lim Yin
[nltk_data]     Shan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Lim Yin
[nltk_data]     Shan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Lim Yin
[nltk_data]     Shan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
url = 'https://drive.google.com/file/d/1z3AN8qN3Sz1UTkDp_Bd8pGi4qqb2rXxu/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
df.dropna(inplace = True)
df.head()

,Text,Class
0,Supply Quality China's EXCLUSIVE dimensions at...,1
1,over. SidLet me know. Thx.,0
2,"Dear Friend,Greetings to you.I wish to accost ...",1
3,MR. CHEUNG PUIHANG SENG BANK LTD.DES VOEUX RD....,1
4,Not a surprising assessment from Embassy.,0


In [3]:
#Generate the confusion matrix
def plot_cm(y_actual,y_pred):
  cf_matrix = confusion_matrix(y_actual,y_pred)

  group_names = ['True Neg','False Pos','False Neg','True Pos']

  group_percentages = ["{0:.2%}".format(value) for value in
                      cf_matrix.flatten()/np.sum(cf_matrix)]

  labels = [f"{v1}\n{v2}" for v1, v2 in
            zip(group_names,group_percentages)]

  labels = np.asarray(labels).reshape(2,2)

  ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

  ax.set_title('Seaborn Confusion Matrix with labels\n\n');
  ax.set_xlabel('\nPredicted Values')
  ax.set_ylabel('Actual Values ');

  ## Ticket labels - List must be in alphabetical order
  ax.xaxis.set_ticklabels(['False','True'])
  ax.yaxis.set_ticklabels(['False','True'])

  ## Display the visualization of the Confusion Matrix.
  plt.show()


# Generate metrics matrix
def metrics(actual,pred):
  print('accuracy: %s%%' % round(accuracy_score(actual,pred)*100,0))
  print('precision: %s%%' % round(precision_score(actual,pred)*100,0))
  print('recall: %s%%' % round(recall_score(actual,pred)*100,0))
  print('f1_score: %s%%' % round(f1_score(actual,pred)*100,0))

In [4]:
def clean_text(df):
    #Converting the text to lower case 
    df['Text'] = df['Text'].astype(str).apply(lambda x: x.lower()) 
    #Remove "=" symbol from data
    df['Text'] = df['Text'].apply(lambda x: x.replace("=",''))
    #Extracting url from the text
    df['Url'] = df['Text'].apply(lambda x: re.findall("http\S+",x))
    #Create new feature called Url_Count
    df['Url_Count'] = df['Url'].apply(lambda x: len(x))
    #Extracting email from the text 
    df['Email'] = df['Text'].apply(lambda x: re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+",x))
    #Create new feature called Email_Count
    df['Email_Count'] = df['Email'].apply(lambda x: len(x))
    #Removing all symbols from the text except the "$" symbol
    df['Text'] = df["Text"].apply(lambda x: re.sub('[^a-z$\s]','',x))
    df["Text_Length"] = df["Text"].apply(lambda x: len(x))
    
    
    # #Drop outliers in Url_Count 
    # range = df['Url_Count'].mean() + df['Url_Count'].std()*3 
    # df = df.loc[(df["Url_Count"] <= range)]
    # #Drop outliers in Email_Count 
    # range = df['Email_Count'].mean() + df['Email_Count'].std()*3 
    # df = df.loc[(df["Email_Count"] <= range)]
    # #Drop outliers in Text_Length
    # range = df['Text_Length'].mean() + df['Text_Length'].std()*3 
    # df = df.loc[(df["Text_Length"] <= range)]

    return df
    
df = clean_text(df)

In [5]:
# Removing stop words part 1: Tokenising the text
df['Text_Tokens'] = df["Text"].apply(lambda x: word_tokenize(x))
# Removing stop words part 2: Removing stopwords
def remove_stop_words(word_tokens):
    nltk_stop_words = stopwords.words('english')
    custom_stop_words = ['.', ',']
    combined_stop_words = nltk_stop_words + custom_stop_words
    filtered_sentence = []
    for w in word_tokens:
        if w not in combined_stop_words:
            filtered_sentence.append(w)
    return (filtered_sentence)
df['Text_Filtered'] = df['Text_Tokens'].apply(lambda x: remove_stop_words(x))

In [6]:
# Lemmatize the list of words
wnl = WordNetLemmatizer()
def lemmatize(s):
    s = [wnl.lemmatize(word) for word in s]
    return s
df['Text_Filtered_Lemmatized'] = df['Text_Filtered'].apply(
    lambda x: lemmatize(x))
# Join the word tokens into strings
df['Text_Filtered_String'] = df['Text_Filtered_Lemmatized'].apply(
    lambda x: ' '.join(x))
df['Url_Present'] = df["Url_Count"].apply(lambda x: 1 if x > 0 else 0)

# TEXT MESSAGE ANALYSIS MODELS
only included NB and SVM models here as they are the highest accuracy ones

In [7]:
# Split dataset to test and train data
X = df["Text_Filtered_String"]
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [8]:
# Model 1 - Multinomial Naive Bayes (With TfidfVectorizer)

vectorizer = TfidfVectorizer()
X_train_idf = vectorizer.fit_transform(X_train)
nb = MultinomialNB()
nb.fit(X_train_idf, y_train)
X_test_idf = vectorizer.transform(X_test)
y_pred = nb.predict(X_test_idf)
metrics(y_test, y_pred)
with open('text_nb.pkl', 'wb') as file:
    pickle.dump(nb, file)

accuracy: 97.0%
precision: 95.0%
recall: 98.0%
f1_score: 97.0%


In [9]:
string = """Hi Yin Shan,

Thank you so much for your confirmation!

Good luck with your interviews :)

Thank you!

Regards
Grace"""
alist = []
alist.append(string)
print(alist)

['Hi Yin Shan,\n\nThank you so much for your confirmation!\n\nGood luck with your interviews :)\n\nThank you!\n\nRegards\nGrace']


In [10]:
test_df = {'Text':[string], 'Class': [0]}
test_df = pd.DataFrame(data=test_df)
test_df = clean_text(test_df)
test_df

,Text,Class,Url,Url_Count,Email,Email_Count,Text_Length
0,hi yin shan\n\nthank you so much for your conf...,0,[],0,[],0,111
